In [65]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
import re
from splinter import Browser
import time

In [138]:
all_links=[]
base_link = "https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao="
all_items=[]
executable_path={'executable_path': 'chromedriver'}
browser=Browser('chrome', **executable_path, headless=False)
for index in range (0,936,24):
    all_links.append(base_link+str(index))
for each_link in all_links: 
    html=get_html(each_link)
    print('visiting {}'.format(each_link))
    one_page=scrape_one(html)
    all_items.append(one_page)
    time.sleep(3)
        #test_response = requests.get(each_link)
        #soup = BeautifulSoup(test_response.text, 'html.parser')

visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=0
26
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=24
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=48
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=72
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=96
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=120
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=144
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=168
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=192
24
visiting https://www.target.com/c/cleaning-supplies-household-essentials/-/N-5xsyy?Nao=216
24
visiting https://www.target.com/c/cleaning-supplies-household-esse

In [129]:
def get_html(link): 
    browser.visit(link)
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(3)
    html=browser.html
    return html

In [137]:
def scrape_one(html):
    one_page=[]
    base_path='http://www.target.com'
    soup=BeautifulSoup(html, 'html.parser')
    results=soup.find_all('div', class_=re.compile('styles__StyledDetailsWrapper'))
    print(len(results))
    for each_result in results:
        item_dict={}
       #finding name, link
        name_and_link=each_result.find('a')
        item_dict['name']=name_and_link['aria-label']
        item_dict['link']=name_and_link['href']
        new_path=base_path+item_dict['link']
        #click of find UPC, Weight and Price in Details
        response=requests.get(new_path)
        item_soup=BeautifulSoup(response.text, 'html.parser')
        item_spec=item_soup.find('div', id='specAndDescript')
        try:
            item_dict['upc']=item_spec.find('b', text='UPC').next_sibling.next_sibling.next_sibling
        except AttributeError:
            pass
        try:
            item_dict['weight']=item_spec.find('b', text='Capacity (Volume):').next_sibling
        except AttributeError:
            pass
        #print('none')
        #find Price
        #price_section=each_result.find('div', class_=re.compile('style__PriceFontSize'))
        #print()
        item_dict['price']=each_result.find('span', attrs= {'data-test': 'product-price'}).text
        #pull ratings
        try:
            item_dict['rating']=each_result.find('span', attrs= {'data-test': 'product-rating'}).text
        except AttributeError:
            pass
        one_page.append(item_dict)
    return one_page

In [405]:
flat_list = []
for sublist in all_items:
    for item in sublist:
        flat_list.append(item)
flat_list

target_items=pd.DataFrame(flat_list)
target_items


,name,link,upc,weight,price,rating
0,Clorox Disinfecting Wipes Bleach Free Cleaning...,/p/clorox-disinfecting-wipes-bleach-free-clean...,044600016658,4.73 ounces,$0.99,4.9 out of 5 stars with 2326 reviews2326
1,Clorox Value Pack Bleach Free Cleaning Scented...,/p/clorox-value-pack-bleach-free-cleaning-scen...,044600302089,2052.0 cubic-in,$5.99 - $9.99,4.8 out of 5 stars with 15862 reviews15862
2,BISSELL Stomp 'n Go 20pk Pet Stain Lifting Pad...,/p/bissell-stomp-n-go-20pk-pet-stain-lifting-p...,011120241402,1.42 ounces,$24.99,4.7 out of 5 stars with 69 reviews69
3,Disinfecting Wipes Lemon and Fresh Scent - 75c...,/p/disinfecting-wipes-lemon-and-fresh-scent-75...,071287838174,68 ounces,$9.99,4.7 out of 5 stars with 80 reviews80
4,Dawn Ultra Original Scent Dishwashing Liquid D...,/p/dawn-ultra-original-scent-dishwashing-liqui...,037000397137,7 fl oz (US),$0.99 - $9.39,4.8 out of 5 stars with 3326 reviews3326
...,...,...,...,...,...,...
891,Seventh Generation Free & Clear,/p/seventh-generation-free-clear/-/A-54388559,NaN,NaN,$2.99 - $17.99,NaN
892,Natural Household Essentials Collection,/p/natural-household-essentials-collection/-/A...,NaN,NaN,$2.79 - $11.99,NaN
893,mDesign Compact Plastic Toilet Bowl Brush and ...,/p/mdesign-compact-plastic-toilet-bowl-brush-a...,192173028346,NaN,$35.99,NaN
894,"Dawn Ultra Original Scent Liquid Dish Soap, Sc...",/p/dawn-ultra-original-scent-liquid-dish-soap-...,829576104703,NaN,$15.99,NaN


In [406]:
#slice ratings to contain only number rating
target_items['rating_score']=target_items['rating'].str[0:3:1]
target_items['link']="www.target"+target_items['link']
target_items['rating_clean']=target_items['rating'].fillna('reviews0')
target_items['rating_count']=[int(sub.split('reviews')[1]) for sub in target_items['rating_clean']]
target_items['price']=target_items['price'].str.lstrip().str[0:6:1]
target_clean=pd.DataFrame(target_items[['name','link','upc','weight','price','rating_score','rating_count',]]).fillna(0)
target_clean.to_csv('data_target.csv')
target_clean


,name,link,upc,weight,price,rating_score,rating_count
0,Clorox Disinfecting Wipes Bleach Free Cleaning...,www.target/p/clorox-disinfecting-wipes-bleach-...,044600016658,4.73 ounces,$0.99,4.9,2326
1,Clorox Value Pack Bleach Free Cleaning Scented...,www.target/p/clorox-value-pack-bleach-free-cle...,044600302089,2052.0 cubic-in,$5.99,4.8,15862
2,BISSELL Stomp 'n Go 20pk Pet Stain Lifting Pad...,www.target/p/bissell-stomp-n-go-20pk-pet-stain...,011120241402,1.42 ounces,$24.99,4.7,69
3,Disinfecting Wipes Lemon and Fresh Scent - 75c...,www.target/p/disinfecting-wipes-lemon-and-fres...,071287838174,68 ounces,$9.99,4.7,80
4,Dawn Ultra Original Scent Dishwashing Liquid D...,www.target/p/dawn-ultra-original-scent-dishwas...,037000397137,7 fl oz (US),$0.99,4.8,3326
...,...,...,...,...,...,...,...
891,Seventh Generation Free & Clear,www.target/p/seventh-generation-free-clear/-/A...,0,0,$2.99,0,0
892,Natural Household Essentials Collection,www.target/p/natural-household-essentials-coll...,0,0,$2.79,0,0
893,mDesign Compact Plastic Toilet Bowl Brush and ...,www.target/p/mdesign-compact-plastic-toilet-bo...,192173028346,0,$35.99,0,0
894,"Dawn Ultra Original Scent Liquid Dish Soap, Sc...",www.target/p/dawn-ultra-original-scent-liquid-...,829576104703,0,$15.99,0,0
